In [1]:
import httpx
import pandas
import tqdm

In [2]:
udp_domain = "udp-kn.de"
realm = "konstanz"
fiware_service = "nodered3"

# I load my secrets from my passwrod manager; you can set them manually

import subprocess
def password(path):
    return subprocess.run(['pass', path], capture_output=True, text=True, check=True).stdout.splitlines()[0]

idm_client = 'api-access'
idm_client_secret = password(f'work/stadtkn/udp/idm/client/{idm_client}')
idm_username = 'patrik.keller@konstanz.de'
idm_password = password(f'work/stadtkn/udp/idm/user/{idm_username}')

In [3]:
def get_token():
    r = httpx.request(
        'POST',
        f'https://idm.{udp_domain}/auth/realms/{realm}/protocol/openid-connect/token',
        data=dict(
            client_id = idm_client,
            client_secret = idm_client_secret,
            grant_type = 'password',
            username = idm_username,
            password = idm_password,
            scope = "api:read api:write api:delete",
        )
    )
    r.raise_for_status()
    return r.json()['access_token']
    
bearer_token = get_token()

In [4]:
df = pandas.read_csv('lake-surface-water-temperature.csv', sep=',', encoding='utf8', parse_dates=['time'])
df

,time,lake_surface_water_temperature,lswt_uncertainty,lswt_quality_level,lswt_obs_instr,lswt_flag_bias_correction,lakeid_CCI,lakeid_GloboLakes
0,1995-06-02 12:00:00,286.45110,0.458000,3.000000,1.000000,1.0,352,352
1,1995-06-08 12:00:00,289.06854,0.481524,3.333333,1.000000,1.0,352,352
2,1995-06-24 12:00:00,285.28000,0.511000,1.000000,1.000000,1.0,352,352
3,1995-06-27 12:00:00,291.51250,0.393062,3.812500,1.000000,1.0,352,352
4,1995-06-30 12:00:00,293.44970,0.247692,4.179487,1.000000,1.0,352,352
...,...,...,...,...,...,...,...,...
5244,2023-12-17 12:00:00,276.01170,0.243333,1.166667,85.333336,NaN,352,352
5245,2023-12-19 12:00:00,278.72247,0.210750,3.500000,32.000000,NaN,352,352
5246,2023-12-27 12:00:00,278.06708,0.209583,3.583333,52.000000,NaN,352,352
5247,2023-12-30 12:00:00,278.48135,0.186614,3.454546,32.000000,NaN,352,352


In [5]:
data_cols = list(df.columns)
data_cols.remove('time')

def map_row(i, row):
    upd = dict(
        id = f"urn:raw:cds:lswt:cci_lake_id:{row.lakeid_CCI}",
        type = "raw_cds_lswt",
        time_index = dict(
            value = row.time.isoformat(),
        ),
    )
    for c in data_cols:
        upd[c] = dict(
            value = row[c],
        )
    return upd

entity_updates = [ map_row(i, r) for i, r in df.iterrows() ]
entity_updates[:2]

[{'id': 'urn:raw:cds:lswt:cci_lake_id:352',
  'type': 'raw_cds_lswt',
  'time_index': {'value': '1995-06-02T12:00:00'},
  'lake_surface_water_temperature': {'value': 286.4511},
  'lswt_uncertainty': {'value': 0.45800003},
  'lswt_quality_level': {'value': 3.0},
  'lswt_obs_instr': {'value': 1.0},
  'lswt_flag_bias_correction': {'value': 1.0},
  'lakeid_CCI': {'value': 352},
  'lakeid_GloboLakes': {'value': 352}},
 {'id': 'urn:raw:cds:lswt:cci_lake_id:352',
  'type': 'raw_cds_lswt',
  'time_index': {'value': '1995-06-08T12:00:00'},
  'lake_surface_water_temperature': {'value': 289.06854},
  'lswt_uncertainty': {'value': 0.48152387},
  'lswt_quality_level': {'value': 3.3333333},
  'lswt_obs_instr': {'value': 1.0},
  'lswt_flag_bias_correction': {'value': 1.0},
  'lakeid_CCI': {'value': 352},
  'lakeid_GloboLakes': {'value': 352}}]

In [6]:
unique_ids = { x['id'] for x in entity_updates }
display(unique_ids)
unique_types = { x['type'] for x in entity_updates }
display(unique_types)

def delete_entity_type(ty):
    r = httpx.request(
        'DELETE',
        f'https://apim.{udp_domain}/gateway/quantumleap/v2/types/{ty}',
        headers = {
            'Authorization' : f"Bearer {get_token()}",
            'Fiware-Service' : fiware_service,
            'Fiware-ServicePath' : '/',
        },
    )
    # r.raise_for_status() # may be 404
    return r

for ty in unique_types:
    delete_entity_type(ty)

{'urn:raw:cds:lswt:cci_lake_id:352'}

{'raw_cds_lswt'}

In [7]:
def post_entity_update_batch(lst):
    assert len(lst) <= 256
    r = httpx.request(
        'POST',
        f'https://apim.{udp_domain}/gateway/quantumleap/v2/notify',
        headers = {
            'Authorization' : f"Bearer {get_token()}",
            'Fiware-Service' : fiware_service,
            'Fiware-ServicePath' : '/',
            'Fiware-TimeIndex-Attribute' : 'time_index',
        },
        json = dict(
            data = lst
        ),
    )
    r.raise_for_status()
    return r

def post_entity_updates(lst):
    batch_size = 256
    for i in tqdm.tqdm(range(0, len(lst), batch_size)):
        post_entity_update_batch(lst[i:i+batch_size])

post_entity_updates(entity_updates)

100%|███████████████████████████████████████████| 21/21 [00:15<00:00,  1.31it/s]
